# Solutions to the Exercises in:  *Arrows and Computation*, by Ross Paterson

Original author: David Banas <capn.freako@gmail.com>  
Original date:   July 29, 2016

Copyright &#169; 2016 David Banas; all rights reserved World wide.


In [1]:
import Data.Time

putStrLn "Notebook last run:"
getCurrentTime


Notebook last run:

2016-10-22 13:23:48.708954 UTC

## Table of Contents <a name="contents"/>

* <a href="#ex1">Exercise 1 - *Arrow* instances for *Reader* and *Writer*.</a>
* <a href="#ex2">Exercise 2 - Attempted *Arrow* instance for *ListMap*.</a>
* <a href="#ex3">Exercise 3 - *Arrow* instance for *StreamtMap*.</a>
* <a href="#ex4">Exercise 4 - Show that a::Arrow &#8905; f::(&#8594;) is a *Functor*.</a>
* <a href="#ex5">Exercise 5 - Verify the *ArrowApply* axioms for pure functions.</a>
* <a href="#ex6">Exercise 6 - Failure of *extensionality* axiom for *ArrowApply* instance of *Auto*.</a>
* <a href="#ex7">Exercise 7 - Define *ArrowChoice* instances for *NonDet*, *State* and the *StreamMap*
type from **Exercise 3**.</a>
  * <a href="#ex7.1">*NonDet*</a>
  * <a href="#ex7.2">*State*</a>
  * <a href="#ex7.3">*StreamMap*</a>
* <a href="#ex8">Exercise 8 - Failure of (⋙) to distribute over (|||), for *Auto* and *StreamMap*</a>
  * <a href="#ex8_auto">*Auto*</a>
  * <a href="#ex8_streammap">*StreamMap*</a>
* <a href="#ex9">Exercise 9 - *Arrow* instance for *Exception*</a>
* <a href="#ex10">Exercise 10 - Proving *first* is a functor, for *Exception*</a>
* <a href="#questions">Unanswered Questions</a>


## Exercise 1 <a name="ex1"/>

Write *Arrow* instances for the following types:

`newtype Reader s a b = R ((s, a)` &#8594; `b)`  
`newtype Writer a b   = W (a` &#8594; `(String, b))`

In the latter case, any monoid could be used in place of String.

In [2]:
import Prelude hiding (pure)

class Arrow a where
    pure  :: (b -> c) -> a b c
    (>>>) :: a b c -> a c d -> a b d
    first :: a b c -> a (b, d) (c, d)

second :: Arrow a => a b c -> a (d, b) (d, c)
second f = pure swap >>> first f >>> pure swap
    where swap ~(x, y) = (y, x)

assoc :: ((a, b), c) -> (a, (b, c))
assoc ~(~(x, y), z) = (x, (y, z))

unassoc :: (a, (b, c)) -> ((a, b), c)
unassoc ~(x, ~(y, z)) = ((x, y), z)

prod :: (a -> a') -> (b -> b') -> (a, b) -> (a', b')
(f `prod` g) ~(x, y) = (f x, g y)

newtype Reader s a b = R {runR :: (s, a) -> b}

newtype Writer a b   = W {runW :: a -> (String, b)}

instance Arrow (Reader s) where
    pure f      = R (\(_, x) -> f x)
    R f >>> R g = R (\(s, x) -> g (s, f (s, x)))
    first (R f) = R (\(s, (x, y)) -> (f (s, x), y))
    
instance Arrow Writer where
    pure f      = W (\x -> ("", f x))
    W f >>> W g = W (\x -> 
        let (s', w) = f x
            (s, y)  = g w
        in (s' ++ s, y))
    first (W f) = W (\(x, y) -> 
        let (s, w) = f x
        in (s, (w, y)))

-- Test axioms.
a1 = R (\(s, x) -> s)
a2 = R (\(s, x) -> x)
a3 = R (\(s, x) -> ())
b1 = W (\x -> ("", x))
b2 = W (\x -> (show x, x + 1))
b3 = W (\x -> ("Hello, World!", x * 2))
x  = ("test", 1)
y  = ("for_first", (2, 3))
z  = ("for_assoc", ((2, 3), 4))
f  = \x -> x + 1
g  = \x -> x * 2
w  = (2, 3)
v = ((2, 3), 4)

-- General utilities
eqR :: (Eq b) => Reader s a b -> Reader s a b -> (s, a) -> Bool
eqR r1 r2 x = runR r1 x == runR r2 x

eqW :: (Eq b) => Writer a b -> Writer a b -> a -> Bool
eqW w1 w2 x = runW w1 x == runW w2 x

-- Functor identity
assertion1 = eqR a1 a' x && eqR a1 a'' x
    where a'  = idA >>> a1
          a'' = a1 >>> idA
          idA = pure id

assertion1w = eqW b1 a' 1 && eqW b1 a'' 1
    where a'  = idA >>> b1
          a'' = b1 >>> idA
          idA = pure id

-- Functor composition
assertion2 = eqR (pure (g . f)) (pure f >>> pure g) x
assertion2w = eqW (pure (g . f)) (pure f >>> pure g) 1

-- Associativity
assertion3 = eqR ((a1 >>> a2) >>> a3) (a1 >>> (a2 >>> a3)) x
assertion3w = eqW ((b1 >>> b2) >>> b3) (b1 >>> (b2 >>> b3)) 1

-- Extension
assertion4 = eqR (first (pure f)) (pure (f `prod` id)) y
assertion4w = eqW (first (pure f)) (pure (f `prod` id)) w

-- Functor
assertion5 = eqR (first (a1 >>> a2)) (first a1 >>> first a2) y
assertion5w = eqW (first (b1 >>> b2)) (first b1 >>> first b2) w

-- Exchange
assertion6 = eqR (first a1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first a1) y
assertion6w = eqW (first b1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first b1) w

-- Unit
assertion7 = eqR (first a1 >>> pure fst) (pure fst >>> a1) y
assertion7w = eqW (first b1 >>> pure fst) (pure fst >>> b1) w

-- Association
assertion8 = eqR (first (first a1) >>> pure assoc) (pure assoc >>> first a1) z
assertion8w = eqW (first (first b1) >>> pure assoc) (pure assoc >>> first b1) v

main :: IO ()
main = do
    print assertion1
    print assertion1w
    print assertion2
    print assertion2w
    print assertion3
    print assertion3w
    print assertion4
    print assertion4w
    print assertion5
    print assertion5w
    print assertion6
    print assertion6w
    print assertion7
    print assertion7w
    print assertion8
    print assertion8w

main


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

<a href="#contents">Back to Contents</a>

## Exercise 2 <a name="ex2"/>

The following is almost an arrow type:

`newtype ListMap ι o = LM ([ι] → [o])`

What goes wrong?

In [3]:
newtype ListMap a b = LM {runLM :: [a] -> [b]}

instance Arrow ListMap where
    pure f        = LM (\xs -> map f xs)
    LM f >>> LM g = LM (\xs -> (g . f) xs)
    first (LM f)  = LM (\ws ->  -- ws :: [(a, b)]
        let (xs, ys) = unzip ws
        in zip (f xs) ys)

-- Test axioms.
-- General utilities
eqLM :: (Eq b) => ListMap a b -> ListMap a b -> [a] -> Bool
eqLM lm1 lm2 xs = runLM lm1 xs == runLM lm2 xs

-- Useful expressions
lm3 = LM (\xs -> map (+1) xs)
lm2 = LM (\xs -> map (*2) xs)
lm1 = LM (\xs -> xs ++ xs)
xs  = [0, 1, 2]
ys  = [(0, "zero"), (1, "one"), (2, "two")]
zs  = [(0, 1), (1, 2), (2, 3)]
zss = [((0, 1), 2), ((1, 2), 3), ((2, 3), 4)]

-- Functor identity
assertion_lm1 = eqLM lm1 a' xs && eqLM lm1 a'' xs
    where a'  = idA >>> lm1
          a'' = lm1 >>> idA
          idA = pure id

-- Functor composition
assertion_lm2 = eqLM (pure (g . f)) (pure f >>> pure g) xs

-- Associativity
assertion_lm3 = eqLM ((lm1 >>> lm2) >>> lm3) (lm1 >>> (lm2 >>> lm3)) xs

-- Extension
assertion_lm4 = eqLM (first (pure f)) (pure (f `prod` id)) ys

-- Functor
assertion_lm5 = eqLM (first (lm1 >>> lm2)) (first lm1 >>> first lm2) ys

-- Exchange
assertion_lm6 = eqLM (first lm1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first lm1) zs

-- Unit
assertion_lm7 = eqLM (first lm1 >>> pure fst) (pure fst >>> lm1) ys

-- Association
assertion_lm8 = eqLM (first (first lm1) >>> pure assoc) (pure assoc >>> first lm1) zss

main :: IO ()
main = do
    print assertion_lm1
    print assertion_lm2
    print assertion_lm3
    print assertion_lm4
    print assertion_lm5
    print assertion_lm6
    print assertion_lm7
    print assertion_lm8

main


True
True
True
True
True
True
False
True

The type of the enclosed function allows us to change the "shape" of the data container (i.e.  *List*, in this case), and that can break some of the properties we need to rely on.

<a href="#contents">Back to Contents</a>

## Exercise 3 <a name="ex3"/>

Define the following as an arrow type:

```
data Stream α = Cons α (Stream α)
newtype StreamMap ι o = SM (Stream ι → Stream o)
```


In [4]:
data Stream a = Cons a (Stream a)

newtype StreamMap a b = SM {runSM :: Stream a -> Stream b}

zipS :: Stream a -> Stream b -> Stream (a, b)
zipS (Cons x xs) (Cons y ys) = Cons (x, y) (zipS xs ys)

unzipS :: Stream (a, b) -> (Stream a, Stream b)
unzipS (Cons z zs) = (Cons (fst z) (fst (unzipS zs)), Cons (snd z) (snd (unzipS zs)))

instance Arrow StreamMap where
    pure f        = SM g
     where g (Cons x xs) = Cons (f x) (g xs)
    SM f >>> SM g = SM (g . f)
    first (SM g)  = SM (\ws ->
     let ys       = g xs
         (xs, zs) = unzipS ws
     in zipS ys zs)

-- Test axioms.
-- General utilities
toList :: Stream a -> [a]
toList (Cons x xs) = x : (toList xs)

eqSM :: (Eq b) => StreamMap a b -> StreamMap a b -> Stream a -> Bool
eqSM sm1 sm2 s = take 100 (toList (runSM sm1 s)) == take 100 (toList (runSM sm2 s))

-- Useful expressions
f (Cons x xs) = Cons (x + 1) (f xs)
sm1 = SM f
g (Cons x xs) = Cons x (Cons x (g xs))
sm2 = SM g
sm3 :: StreamMap a a
sm3 = pure id
sms = [sm1, sm2, sm3]

s1 = Cons 0 (s1)
s2 = Cons 0 (f s2)
ss = [s1, s2]

f2 = \x -> 1
f3 = \x -> x + 1
f4 = \x -> x * 2
fs = [f2, f3, f4]

-- Functor identity
idSM :: StreamMap a a
idSM = pure id

assertion_sm1 = and [eqSM sm (idSM >>> sm) s && eqSM sm (sm >>> idSM) s | sm <- sms, s <- ss]

-- Functor composition
assertion_sm2 = and [eqSM (pure (g . f)) (pure f >>> pure g) s | f <- fs, g <- fs, s <- ss]

-- Associativity
assertion_sm3 = and [eqSM ((sm1 >>> sm2) >>> sm3) (sm1 >>> (sm2 >>> sm3)) s | s <- ss]

-- Extension
assertion_sm4 = and [eqSM (first (pure f)) (pure (f `prod` id)) (zipS s1 s2) | f <- fs]

-- Functor
assertion_sm5 = and [eqSM (first (sm >>> sm')) (first sm >>> first sm') (zipS s1 s2) | sm <- sms, sm' <- sms]

-- Exchange
assertion_sm6 = and [eqSM (first sm >>> pure (id `prod` f)) (pure (id `prod` f) >>> first sm) (zipS s1 s2) | sm <- sms, f <- fs]

-- Unit
assertion_sm7 = and [eqSM (first sm >>> pure fst) (pure fst >>> sm) (zipS s1 s2) | sm <- sms]

-- Association
assertion_sm8 = and [eqSM (first (first sm) >>> pure assoc) (pure assoc >>> first sm) (zipS (zipS s1 s2) s2) | sm <- sms]

main :: IO ()
main = do
    putStrLn "Streams used (first 10 elements):"
    putStr   "\ts1: "
    print $ take 10 $ toList s1
    putStr   "\ts2: "
    print $ take 10 $ toList s2
    putStrLn ""
    putStrLn "Assertion values:"
--    print res
    print assertion_sm1
    print assertion_sm2
    print assertion_sm3
    print assertion_sm4
    print assertion_sm5
    print assertion_sm6
    print assertion_sm7
    print assertion_sm8

main


Streams used (first 10 elements):
	s1: [0,0,0,0,0,0,0,0,0,0]
	s2: [0,1,2,3,4,5,6,7,8,9]

Assertion values:
True
True
True
True
True
True
True
True

<a href="#contents">Back to Contents</a>

## Exercise 4 <a name="ex4"/>

Show that the following is a Functor.

```
(<*) :: Arrow a => a b c -> (b' -> c') -> a (b, b') (c, c')
f <* g = first f >>> pure (id × g)
```

Note: I added "pure".

From a private e-mail exchange with Ross:

To show:

```
pure id <* id = pure id

(f >>> f') <* (g' . g) = (f <* g) >>> (f' <* g')
```

My solutions:

`pure id <* id                       =` {definition of `(<*)`}  
`first (pure id) >>> pure (id x id)  =` {extension}  
`pure (id x id)  >>> pure (id x id)  =` {functor-composition}  
`pure ((id x id) . (id x id))        =` {(x) is a functor in two arguments.}  
`pure ((id . id) x (id . id))        =` {idempotency of *id*}  
`pure (id x id)                      =` {by simple inspection}  
`idA                                 =` {functor-identity}  
`pure id`  
&#9633;

`(f >>> f') <* (g' . g)                                        =` {definition of `(<*)`}  
`first (f >>> f') >>> pure (id x (g' . g))                     =` {idempotency of *id*}  
`first (f >>> f') >>> pure ((id . id) x (g' . g))              =` {(x) is a functor in two arguments.}  
`first (f >>> f') >>> pure ((id x g') . (id x g))              =` {functor-composition}  
`first (f >>> f') >>> pure (id x g) >>> pure (id x g')         =` {functor}  
`first f >>> first f' >>> pure (id x g) >>> pure (id x g')     =` {exchange}  
`first f >>> pure (id x g) >>> first f' >>> pure (id x g')     =` {association?}  
`(first f >>> pure (id x g)) >>> (first f' >>> pure (id x g')) =` {definition of `(<*)`}  
`(f <* g) >>> (f' <* g')`  
&#9633;


<a href="#contents">Back to Contents</a>

## Exercise 5 <a name="ex5"/>

Verify the *ArrowApply* axioms for pure functions. 

To show:

```
pure ((⋙ h) × id) ⋙ app = app ⋙ h
pure (mkPair × id) ⋙ app = pure id
mkPair f           ⋙ app = f

mkPair :: Arrow α ⇒ β → α γ (β, γ)
mkPair b = pure (λc → (b, c))
```

Given:

```
instance Arrow (→) where  
  pure f  = f  
  f ⋙ g  = g · f  
  first f = f × id

instance ArrowApply (→) where
  app  ̃(f, c) = f c
```

We have:

`pure ((⋙ h) × id) ⋙ app              =` {definitions of *pure* and (&#8921;), for (→)}  
`app . ((⋙ h) × id)                   =` {definitions of *app* and (&#8921;), for (→)}  
`(λ(f, c) → f c) . ((λg → h . g) × id) =` {definitions of composition and application}  
`λ(f', a) → (h . f') a                 =` {definition of composition}  
`λ(f', a) → h (f' a)                   =` {(Not sure how to justify this step.)}  
`h . (λ(f', a) → f' a)                 =` {definitions of *app* and (&#8921;), for (→)}  
`app ⋙ h`  
&#9633;

`pure (mkPair × id) ⋙ app      =` {definitions of *pure* and (&#8921;), for (→)}   
`app . (mkPair × id)            =` {definitions of (×), *mkPair*, and *id*}  
`λ(b, d) → app (λc → (b, c), d) =` {definition of *app* for (→)}  
`λ(b, d) → (b, d)               =` {definition of *id*}  
`id                             =` {definition of *pure* for (→)}  
`pure id`  
&#9633;

`mkPair f ⋙ app     =` {definition of (⋙), for (→)}  
`app . mkPair f      =` {definition of *mkPair*}  
`app . (λx → (f, x)) =` {(reverse of unsure step, above.}  
`λx → app (f, x)     =` {definition of *app*, for (→)}  
`λx → f x            =` {beta (eta?) reduction}  
`f`  
&#9633;


<a href="#contents">Back to Contents</a>

## Exercise 6 <a name="ex6"/>

The following instance has the correct type:

```
instance ArrowApply Auto where
app = pure (λ (A f , x ) → fst (f x ))
```

Show that the *extensionality* axiom fails for this definition.  
(**extensionality**: mkPair f ⋙ app = f)

```
instance Arrow Auto where
  pure f = A (λb → (f b, pure f))
  
  A f ≫ A g = A(λb →
    let (c, f′) = f b
        (d, g′) = g c
    in (d, f′ ≫ g′))

  first (A f) = A (λ(b, d) →
    let (c, f′) = f b
    in ((c, d), first f′))
```

`mkPair f ⋙ app                                                                    =`  {definition of *mkPair*}  
`pure (λx → (f, x)) ⋙ app                                                          =`  {definition of *app*, for *Auto*}  
`pure (λx → (f, x)) ⋙ pure (λ(A g, y) → fst (g y))                                 =`  {definition of *pure*, for *Auto*}  
`A (λb → ((λx → (f, x)) b, pure (λx → (f, x)))) ⋙ pure (λ(A g, y) → fst (g y))     =`  {application}  
`A (λb → ((f, b), pure (λx → (f, x)))) ⋙ pure (λ(A g, y) → fst (g y))              =`  {definition of *pure*, for *Auto*}  
`A (λb → ((f, b), pure (λx → (f, x)))) ⋙
  A (λ(A h, z) → ((λ(A g, y) → fst (g y)) (A h, z), pure (λ(A g, y) → fst (g y)))) =`  {application}  
`A (λb → ((f, b), pure (λx → (f, x)))) ⋙
  A (λ(A h, z) → (fst (h z), pure (λ(A g, y) → fst (g y))))                        =`  {definition of (⋙), for *Auto*}  
`A (λb → let (c@(A h, z), f') = ((f, b), pure (λx → (f, x)))
             (d,          g') = (fst (h z), pure (λ(A g, y) → fst (g y)))
         in  (d, f' >>> g'))                                                       =`  {evaluation}  
`A (λb → (fst (h b), pure (λx → (f, x)) ⋙ pure (λ(A g, y) → fst (g y))))`  
&#8800; `f@(A h)`  
&#9633;


<a href="#contents">Back to Contents</a>

## Exercise 7 <a name="ex7"/>

Define *ArrowChoice* instances for *NonDet*, *State* and the *StreamMap*
type from **Exercise 3**.

### *NonDet* <a name="ex7.1"/>

Taking the *Arrow* instance from the text and validating it...

In [5]:
newtype NonDet a b = ND {runND :: a -> [b]}

instance Arrow NonDet where
    pure f        = ND (\x -> [f x])
    ND f >>> ND g = ND (\b -> [d | c <- f b, d <- g c])
    first (ND f)  = ND (\(b, d) -> [(c, d) | c <- f b])

eqND :: (Eq b) => NonDet a b -> NonDet a b -> a -> Bool
eqND nd1 nd2 x = runND nd1 x == runND nd2 x

-- xs = ((), 0, 1, 'a', "Hello, World!")
xs = [0, 1, 2]
ys = [(x, y) | x <- xs, y <- xs]
zs = [(p, x) | p <- ys, x <- xs]

nd1 = ND (\x -> [])
nd2 = ND (\x -> [()])
nd3 = ND (\x -> [x])
nd4 = ND (\x -> [(), x])
nd5 = ND (\x -> [x, ()])
nd6 = ND (\x -> [x, x])
-- nds = [nd1, nd2, nd3, nd4, nd5, nd6]
nds = [nd1, nd3, nd6]

f1  = \x -> ()
f2  = \x -> 1
f3  = \x -> x
f4  = \x -> x + 1
f5  = \x -> x * 2
f6  = \x -> []
f7  = \x -> [()]
f8  = \x -> [x]
f9  = \x -> [(), x]
f10 = \x -> [x, ()]
f11 = \x -> [x, x]
-- fs = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11]
fs = [f2, f3, f4, f5]
-- fs = [f6, f7, f8, f9, f10, f11]

-- Functor identity
assertion_nd1 = and [ let a  = idA >>> nd
                          a' = nd  >>> idA
                      in eqND nd a x && eqND nd a' x | nd <- nds, x <- xs]
    where idA = pure id

-- Functor composition
assertion_nd2 = and [ eqND (pure (g . f)) (pure f >>> pure g) x | g <- fs, f <- fs, x <- xs ]

-- Associativity
assertion_nd3 = and [ eqND ((nd1 >>> nd2) >>> nd3) (nd1 >>> (nd2 >>> nd3)) x | nd1 <- nds, nd2 <- nds, nd3 <- nds, x <- xs ]

-- Extension
assertion_nd4 = and [ eqND (first (pure f)) (pure (f `prod` id)) y | f <- fs, y <- ys ]

-- Functor
assertion_nd5 = and [ eqND (first (nd >>> nd')) (first nd >>> first nd') y | nd <- nds, nd' <- nds, y <- ys ]

-- Exchange
assertion_nd6 = and [ eqND (first nd >>> pure (id `prod` f)) (pure (id `prod` f) >>> first nd) y | nd <- nds, f <- fs, y <- ys ]

-- Unit
assertion_nd7 = and [ eqND (first nd >>> pure fst) (pure fst >>> nd) y | nd <- nds, y <- ys ]

-- Association
assertion_nd8 = and [ eqND (first (first nd) >>> pure assoc) (pure assoc >>> first nd) z | nd <- nds, z <- zs ]

main :: IO ()
main = do
    print assertion_nd1
    print assertion_nd2
    print assertion_nd3
    print assertion_nd4
    print assertion_nd5
    print assertion_nd6
    print assertion_nd7
    print assertion_nd8

main


True
True
True
True
True
True
True
True

Now, define the *ArrowChoice* instance and validate it...

In [6]:
class Arrow a => ArrowChoice a where
    left :: a b c -> a (Either b d) (Either c d)
    
instance ArrowChoice NonDet where
    left (ND f) = ND (g)
     where g (Left  x) = [Left y | y <- f x]
           g (Right e) = [Right e]

**extension**: left (pure f) = pure (f ⊕ id)  

`left (pure f)                                  =`  {definition of *pure*, for *NonDet*}  

`left (ND (λx → [f x]))                         =`  {definition of *left* for *NonDet*)  

`ND (λp → case p of  Left x  → [Left (f x)]  
                    Right e → [Right e])       =`  {definition of *pure* for *NonDet*}  

`pure (λp → case p of  Left x  → Left (f x)  
                      Right e → Right e)       =`  {definition of *id*}  

`pure (λp → case p of  Left x  → Left (f x)  
                      Right e → Right (id e))  =`  {definition of ⊕}  

`pure (f ⊕ id)`  
&#9633;

**functor**: left (f ⋙ g) = left f ⋙ left g  

`left (ND f ⋙ ND g)                                   =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}  

`left (ND (λb → [d | c ← f b, d ← g c]))             =` &nbsp; &nbsp; {definition of *left* for *NonDet*}   

`ND (λp → case p of  
  Left x → [Left y | y ← [d | c ← f x, d ← g c]]  
 Right e → [Right e])                                 =` &nbsp; &nbsp; {evaluation/inspection of case alternatives}
 
`ND (λp → [d | p' ← case p of  Left x  → [Left y  | y  ← f x ]  
                              Right e  → [Right e ],  
              d  ← case p' of  Left c → [Left y' | y' ← g c ]  
                              Right e' → [Right e']]) =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}  

`ND (λp → case p of    Left x  → [Left y  | y ← f x  ]  
                     Right e  → [Right e ])  
 ⋙  
 ND (λp → case p of   Left x' → [Left y' | y' ← g x']  
                     Right e' → [Right e'])           =` &nbsp; &nbsp; {definition of *left* for *NonDet*}  

left (ND f) ⋙ left (ND g)`  
&#9633;

(Note: the above works, because *NonDet* is guaranteed to never swap *Left*s and *Right*s.
That is, it never changes the shape/nature of the *container*.)


**exchange**: left f ⋙ pure (id ⊕ g) = pure (id ⊕ g) ⋙ left f

`left (ND f) ⋙ pure (id ⊕ g)                                  =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`left (ND f) ⋙ ND (λp → [(id ⊕ g) p])                         =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`ND (λp → case p of  Left x → [Left y | y ← f x]  
                   Right e → [Right e])  
 ⋙ ND (λp → [(id ⊕ g) p])                                    =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}

`ND ( λp → [ d |  
  p' ← case p of  Left x → [ Left y  | y  ← f x ]  
                  Right e → [ Right e ],  
  d  ← [(id ⊕ g) p'] ] )                                     =` &nbsp; &nbsp; {definitions of (⊕) and *id*}  

`ND ( λp → [ d |  
  p' ← case p  of  Left x  → [ Left y  | y  ← f x ]  
                   Right e  → [ Right e ],  
  d  ← case p' of  Left x' → [Left x']  
                   Right e' → [Right (g e')] ] )              =` &nbsp; &nbsp; {inspection/evaluation of alternatives}  

`ND ( λp → [ d | d ← case p of  Left x → [ Left y  | y  ← f x ]  
                               Right e → [ Right (g e) ]      =` &nbsp; &nbsp; {inspection/evaluation of alternatives}  

`ND ( λp → [ d | p' ← case p  of  Left x  → [ Left x ]  
                                 Right e  → [ Right (g e) ],  
                d  ← case p' of  Left x' → [ Left y | y ← f x' ]
                                 Right e' → [ Right e ] )     =` &nbsp; &nbsp; {definitions of (⊕) and *id*}

`ND ( λp → [ d | p' ← [ (id ⊕ g) p ],  
                d  ← case p' of  Left x → [ Left y | y ← f x ]
                                 Right e → [ Right e ] )      =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}


`ND ( λp → [ (id ⊕ g) p ] )  
  ⋙  ND ( λp → case p of  Left x → [ y | y ← f x ]  
                          Right e → [ Right e ] )             =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`ND ( λp → [ (id ⊕ g) p ] ) ⋙ left (ND f)                     =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`pure (id ⊕ g) ⋙ left (ND f)`  
&#9633;


**unit**: pure Left ⋙ left (ND f) = ND f ⋙ pure Left

`pure Left ⋙ left f                                    =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`ND ( λx → [Left x] ) ⋙ left f                         =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`ND ( λx → [Left x] ) ⋙ ND ( λp →  
  case p of Left x → [Left y | y <- (f x)],  
           Right e → [Right e] )                       =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}

`ND ( λx → [ d | p ← [Left x],  
                d ← case p of Left x → [Left y | y ← f x],  
                              Right e → [Right e] ] )  =` &nbsp; &nbsp; {evaluation/inspection of alternatives}




`ND ( λx → [Left y | y ← f x] )                         =` &nbsp; &nbsp; {equivalent by evaluation/inspection}

`ND ( λx → [ d | c ← f x,  
                d ← [Left c]] )                        =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}

`ND f ⋙ ND ( λx → [Left x] )                            =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`ND f ⋙ pure Left`  
&#9633;


**association**: left (left f) ⋙ pure assocsum = pure assocsum ⋙ left f

```
assocsum :: Either (Either α β) γ → Either α (Either β γ)  
assocsum (Left (Left a))  = Left a  
assocsum (Left (Right b)) = Right (Left b)  
assocsum (Right c)        = Right (Right c)
```

`left (left (ND f)) ⋙ pure assocsum                       =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`left (left (ND f)) ⋙ ND ( λp → [assocsum p] )            =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`left (ND ( λp → case p of  Left x → [Left y | y ← f x]  
                           Right e → [Right e] )) ⋙  
  ND ( λp → [assocsum p] )                                =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`ND ( λp' → case p' of  Left p  → [ Left c | c ← 
                                    case p of Left x → [Left y | y ← f x]
                                             Right e → [Right e] ]
                       Right e' → [Right e'] ) ⋙
  ND ( λp → [assocsum p] )                                =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}

`ND ( λp' → [ d | q ← case p' of  Left p  → [ Left c | c ← 
                                    case p of Left x → [Left y | y ← f x]
                                             Right e → [Right e] ]
                                 Right e' → [Right e']
                  d ← [assocsum q] ] )                   =` &nbsp; &nbsp; {Evaluating all possible cases of `p'`:}

*`p' = Left (Left x)`:*  
`=> [ assocsum (Left (Left y)) | y ← f x ] = [ Left y | y ← f x ]`

*`p' = Left (Right e)`:*  
`=> [ assocsum (Left (Right e)) ]           = [ Right (Left e) ]`

*`p' = Right e`:*  
`=> [ assocsum (Right e') ]                 = [ Right (Right e') ]`

The 3 results, above, are also what we get, when evaluating the equation, below, for all possibilities of `p`.

`ND ( λp → [ d | p' ← [assocsum p]
                 d  ← case p' of Left x → [Left y | y ← f x]
                                 Right e → [Right e] ] ) =` &nbsp; &nbsp; {definition of (⋙) for *NonDet*}

`ND ( λp → [assocsum p] ) ⋙
  ND ( λp' → case p' of Left x → [Left y | y ← f x]
                       Right e → [Right e] )             =` &nbsp; &nbsp; {definition of *left* for *NonDet*}

`ND ( λp → [assocsum p] ) ⋙ left (ND f)                  =` &nbsp; &nbsp; {definition of *pure* for *NonDet*}

`pure assocsum ⋙ left (ND f)`  
&#9633;


<a href="#contents">Back to Contents</a>

### *State* <a name="ex7.2"/>

Taking the *Arrow* instance from the text and validating it...

In [7]:
newtype State s a b = ST {runST :: (s, a) -> (s, b)}

instance Arrow (State s) where
    pure f        = ST (id `prod` f)
    ST f >>> ST g = ST (g . f)
    first (ST f)  = ST (assoc . (f `prod` id) . unassoc)

eqST :: (Eq s, Eq b) => State s a b -> State s a b -> s -> a -> Bool
eqST st1 st2 s x = runST st1 (s, x) == runST st2 (s, x)

-- xs = ((), 0, 1, 'a', "Hello, World!")
xs = [0, 1, 2]
ys = [(x, y) | x <- xs, y <- xs]
zs = [(p, x) | p <- ys, x <- xs]

f1  = \x -> ()
f2  = \x -> 1
f3  = \x -> x
f4  = \x -> x + 1
f5  = \x -> x * 2
f6  = \x -> []
f7  = \x -> [()]
f8  = \x -> [x]
f9  = \x -> [(), x]
f10 = \x -> [x, ()]
f11 = \x -> [x, x]
-- fs = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11]
fs = [f2, f3, f4, f5]
-- fs = [f6, f7, f8, f9, f10, f11]

sts = [ST (f1 `prod` f2) | f1 <- fs, f2 <- fs]

-- Functor identity
assertion_st1 = and [ let a  = idA >>> st
                          a' = st  >>> idA
                      in eqST st a s x && eqST st a' s x | st <- sts, s <- xs, x <- xs]
    where idA = pure id

-- Functor composition
assertion_st2 = and [ eqST (pure (g . f)) (pure f >>> pure g) s x | g <- fs, f <- fs, s <- xs, x <- xs ]

-- Associativity
assertion_st3 = and [ eqST ((st1 >>> st2) >>> st3) (st1 >>> (st2 >>> st3)) s x |
                        st1 <- sts, st2 <- sts, st3 <- sts, s <- xs, x <- xs ]

-- Extension
assertion_st4 = and [ eqST (first (pure f)) (pure (f `prod` id)) s y | f <- fs, s <- xs, y <- ys ]

-- Functor
assertion_st5 = and [ eqST (first (st >>> st')) (first st >>> first st') s y |
                        st <- sts, st' <- sts, s <- xs, y <- ys ]

-- Exchange
assertion_st6 = and [ eqST (first st >>> pure (id `prod` f)) (pure (id `prod` f) >>> first st) s y |
                        st <- sts, f <- fs, s <- xs, y <- ys ]

-- Unit
assertion_st7 = and [ eqST (first st >>> pure fst) (pure fst >>> st) s y | st <- sts, s <- xs, y <- ys ]

-- Association
assertion_st8 = and [ eqST (first (first st) >>> pure assoc) (pure assoc >>> first st) s z | st <- sts, s <- xs, z <- zs ]

main :: IO ()
main = do
    print assertion_st1
    print assertion_st2
    print assertion_st3
    print assertion_st4
    print assertion_st5
    print assertion_st6
    print assertion_st7
    print assertion_st8

main


True
True
True
True
True
True
True
True

Now, define the *ArrowChoice* instance and validate it...

In [8]:
instance ArrowChoice (State s) where
    left (ST f) = ST ( \(s, p) -> case p of
                                    Left  x -> ( let (s', y) = f (s, x)
                                                 in  (s', Left y) )
                                    Right e -> (s, Right e) )

-- **extension**: left (pure f) = pure (f ⊕ id)
(<+>) :: (a -> b) -> (e -> e') -> Either a e -> Either b e'
f <+> g = \p -> case p of
                  Left  x -> Left  (f x)
                  Right e -> Right (g e)
                  
ps = map Left xs ++ (map Right xs)

assertion_st11 = and [ eqST (left (pure f)) (pure (f <+> id)) s p |
                         f <- fs, s <- xs, p <- ps ]

-- functor: left (f ⋙ g) = left f ⋙ left g
assertion_st12 = and [ eqST (left (pure f >>> (pure g))) (left (pure f) >>> (left (pure g))) s p |
                         f <- fs, g <- fs, s <- xs, p <- ps ]

-- exchange: left f ⋙ pure (id ⊕ g) = pure (id ⊕ g) ⋙ left f
assertion_st13 = and [ eqST (left (pure f) >>> (pure (id <+> g))) (pure (id <+> g) >>> (left (pure f))) s p |
                         f <- fs, g <- fs, s <- xs, p <- ps ]

-- unit: pure Left ⋙ left f = f ⋙ pure Left
assertion_st14 = and [ eqST (pure Left >>> (left (pure f))) (pure f >>> (pure Left)) s x |
                         f <- fs, s <- xs, x <- xs ]

-- association: left (left f) ⋙ pure assocsum = pure assocsum ⋙ left f
assocsum :: Either (Either α β) γ -> Either α (Either β γ)  
assocsum (Left (Left a))  = Left a  
assocsum (Left (Right b)) = Right (Left b)  
assocsum (Right c)        = Right (Right c)

pps = map Left ps ++ (map Right xs)
assertion_st15 = and [ eqST (left (left (pure f)) >>> (pure assocsum)) (pure assocsum >>> (left (pure f))) s pp |
                         f <- fs, s <- xs, pp <- pps ]

-- distribution: first (left f ) ≫ pure distr = pure distr ≫ left (first f ) 
distr :: (Either α β, γ) -> Either (α, γ) (β, γ)
distr (Left  a, c) = Left  (a, c)
distr (Right b, c) = Right (b, c)

prs = zip ps (xs ++ xs)
assertion_st16 = and [ eqST (first (left (pure f)) >>> (pure distr)) (pure distr >>> (left (first (pure f)))) s pr |
                         f <- fs, s <- xs, pr <- prs ]

main :: IO ()
main = do
    print assertion_st11
    print assertion_st12
    print assertion_st13
    print assertion_st14
    print assertion_st15
    print assertion_st16

main


True
True
True
True
True
True

<a href="#contents">Back to Contents</a>

### *StreamMap* <a name="ex7.3"/>

Using the *Arrow* instance from <a href="#ex3">Exercise 3</a>...

In [9]:
fix f = f (fix f)

instance ArrowChoice StreamMap where
    left sm@(SM f) = SM ( \st@(Cons p ps) ->
      case p of
        Left  x -> let st'@(Cons q _) = f (Cons x (fix (Cons x)))
                   in Cons (Left q) (runSM (left sm) ps)
        Right e -> Cons (Right e) (runSM (left sm) ps) )
        
-- extension: left (pure f) = pure (f ⊕ id)
instance Functor Stream where
    fmap f (Cons x xs) = Cons (f x) (fmap f xs)

interleaveS :: Stream a -> Stream a -> Stream a
interleaveS (Cons x xs) (Cons y ys) = Cons x (Cons y (interleaveS xs ys))

(***) :: Arrow a => a b c -> a b' c' -> a (b, b') (c, c')
f *** g = first f >>> (second g)

(&&&) :: Arrow a => a b c -> a b c' -> a b (c, c')
f &&& g = pure dup >>> (f *** g)
    where dup b = (b, b)

instance Arrow (->) where
    pure f  = f
    f >>> g = g . f
    first f = f `prod` id

sps = map (uncurry interleaveS . ((fmap Left) &&& (fmap Right))) ss

assertion_sm11 = and [eqSM (left (pure f)) (pure (f <+> id)) sp | f <- fs, sp <- sps]

-- functor: left (f ⋙ g) = left f ⋙ left g
assertion_sm12 = and [eqSM (left ((pure f) >>> (pure g))) (left (pure f) >>> (left (pure g))) sp |
                        f <- fs, g <- fs, sp <- sps]

-- exchange: left f ⋙ pure (id ⊕ g) = pure (id ⊕ g) ⋙ left f
assertion_sm13 = and [eqSM (left (pure f) >>> (pure (id <+> g))) (pure (id <+> g) >>> (left (pure f))) sp |
                        f <- fs, g <- fs, sp <- sps]

-- unit: pure Left ⋙ left f = f ⋙ pure Left
assertion_sm14 = and [eqSM (pure Left >>> (left (pure f))) (pure f >>> (pure Left)) s |
                        f <- fs, s <- ss]

-- association: left (left f) ⋙ pure assocsum = pure assocsum ⋙ left f
spps = map (interleaveS (fmap Right s2) . (fmap Left)) sps
assertion_sm15 = and [eqSM (left (left (pure f)) >>> (pure assocsum)) (pure assocsum >>> (left (pure f))) spp |
                        f <- fs, g <- fs, spp <- spps]

-- distribution: first (left f ) ≫ pure distr = pure distr ≫ left (first f ) 
sprs = [zipS sp s | sp <- sps, s <- ss]
assertion_sm16 = and [eqSM (first (left (pure f)) >>> (pure distr)) (pure distr >>> (left (first (pure f)))) spr |
                        f <- fs, spr <- sprs]

main :: IO ()
main = do
    putStrLn "spps (first 10 elements):"
    sequence $ map (print . take 10 . toList) spps
    putStrLn ""

    putStrLn "sprs (first 10 elements):"
    sequence $ map (print . take 10 . toList) sprs
    putStrLn ""

    putStrLn "Assertions:"
    print assertion_sm11
    print assertion_sm12
    print assertion_sm13
    print assertion_sm14
    print assertion_sm15
    print assertion_sm16

main


spps (first 10 elements):
[Right 0,Left (Left 0),Right 1,Left (Right 0),Right 2,Left (Left 0),Right 3,Left (Right 0),Right 4,Left (Left 0)]
[Right 0,Left (Left 0),Right 1,Left (Right 0),Right 2,Left (Left 1),Right 3,Left (Right 1),Right 4,Left (Left 2)]

sprs (first 10 elements):
[(Left 0,0),(Right 0,0),(Left 0,0),(Right 0,0),(Left 0,0),(Right 0,0),(Left 0,0),(Right 0,0),(Left 0,0),(Right 0,0)]
[(Left 0,0),(Right 0,1),(Left 0,2),(Right 0,3),(Left 0,4),(Right 0,5),(Left 0,6),(Right 0,7),(Left 0,8),(Right 0,9)]
[(Left 0,0),(Right 0,0),(Left 1,0),(Right 1,0),(Left 2,0),(Right 2,0),(Left 3,0),(Right 3,0),(Left 4,0),(Right 4,0)]
[(Left 0,0),(Right 0,1),(Left 1,2),(Right 1,3),(Left 2,4),(Right 2,5),(Left 3,6),(Right 3,7),(Left 4,8),(Right 4,9)]

Assertions:
True
True
True
True
True
True

## Exercise 8 <a name="ex8"/>

Show that the equation

`(f ||| g) ⋙ h = (f ⋙ h) ||| (g ⋙ h)`

fails for the *Auto* and *StreamMap* arrows.

### *Auto* <a name="ex8_auto"/>

Useful stuff, copied from the text:

```
A f ≫ A g = A ( λb →
    let (c, f′) = f b
        (d, g′) = g c
    in  (d, f′ ≫ g′) )
```

```
instance ArrowChoice Auto where
    left (A f) = A lf
        where lf (Left b)  = let (c, f′) = f b
                             in (Left c, left f′)
              lf (Right d) = (Right d, left (A f))
```

```
(<+>) :: ArrowChoice α ⇒ α β γ → α β′ γ′ → α (Either β β′) (Either γ γ′)
f <+> g = left f ≫ right g
```

```
(|||) :: ArrowChoice α ⇒ α β δ → α γ δ → α (Either β γ) δ
f ||| g = f <+> g ≫ pure untag
where untag (Left x)  = x
      untag (Right y) = y
```

`(A f ||| A g) ⋙ A h                          =` &nbsp; &nbsp; {definition of (|||)}

`(A f <+> A g ⋙ pure untag) ⋙ A h             =` &nbsp; &nbsp; {definition of (<+>)}

`left (A f) ⋙ right (A g) ⋙ pure untag ⋙ A h =` &nbsp; &nbsp; {definition of *left*, for *Auto*}

`A ( λp → case p of
            Left  b → let (c, f') = f b
                      in (Left c, left f')
            Right d → (Right d, left (A f))
   ) ⋙ right (A g) ⋙ pure untag ⋙ A h        =` &nbsp; &nbsp; {definition of *right*, for *Auto*}

`A ( λp → case p of
            Left  b → let (c, f') = f b
                      in (Left c, left f')
            Right d → (Right d, left (A f))
   ) ⋙ A ( λq → case q of
            Right b → let (c, f') = f b
                      in (Right c, right f')
            Left  d → (Left d, right (A f))
          )
     ⋙ pure untag ⋙ A h                      =` &nbsp; &nbsp; {definition of (⋙), for *Auto*}

Ooof! My head is smoking. I think I'll use the code approach, instead...  
(Probably, a better approach, in general, when I'm being asked to *dis*prove something,  
since that only requires exhibiting a single counterexample.)

In [10]:
newtype Auto a b = A {runA :: a -> (b, Auto a b)}

instance Arrow Auto where
    pure f      = A (\b -> (f b, pure f))
    A f >>> A g = A (\b -> let (c, f') = f b
                               (d, g') = g c
                           in  (d, f' >>> g'))
    first (A f) = A (\(b, d) -> let (c, f') = f b
                                in  ((c, d), first f'))

instance ArrowChoice Auto where
    left (A f) = A lf
        where lf (Left b)  = let (c, f') = f b
                             in (Left c, left f')
              lf (Right d) = (Right d, left (A f))

(<+>) :: ArrowChoice a => a b c -> a b' c' -> a (Either b b') (Either c c')
f <+> g = left f >>> right g

(|||) :: ArrowChoice a => a b d -> a c d -> a (Either b c) d
f ||| g = f <+> g >>> pure untag
    where untag (Left x)  = x
          untag (Right y) = y

right :: ArrowChoice a => a b c -> a (Either d b) (Either d c)
right f = pure mirror >>> left f >>> pure mirror
    where mirror (Left  x) = Right x
          mirror (Right y) = Left  y

proc_pair :: Auto a b -> (a, a) -> (b, b)
proc_pair a (x1, x2) = let (y1, a')  = runA a x1
                           (y2, a'') = runA a' x2
                       in  (y1, y2)
                       
eqA :: (Eq b) => Auto a b -> Auto a b -> (a, a) -> Bool
eqA a1 a2 pr = proc_pair a1 pr == proc_pair a2 pr

a1 = pure (\x -> 12)
a2 = pure id
a3 = pure (+ 1)
a4 = pure (* 2)
a5 = A (\x -> (x + 1, pure id))
as  = [a1, a2, a3, a4]
as' = [a1, a2, a3, a4, a5]

xs  = [0..9]
ps  = map Left xs ++ (map Right xs)
prs = zip ps (tail ps)

-- This one uses as.
assertion_a1 = and [eqA ((a ||| a') >>> a'') ((a >>> a'') ||| (a' >>> a'')) pr
                    | a <- as, a' <- as, a'' <- as, pr <- prs]
-- This one uses as'.
assertion_a2 = and [eqA ((a ||| a') >>> a'') ((a >>> a'') ||| (a' >>> a'')) pr
                    | a <- as', a' <- as', a'' <- as', pr <- prs]

main :: IO ()
main = do
    putStrLn "Assertions:"
    print assertion_a1
    print assertion_a2

main


Assertions:
True
False

When only *pure* is used to create the Automata, the equality holds, because the function computed doesn't change.  
But, with an automaton that changes the computation, the distribution of sequencing over the choice operator breaks.

<a href="#contents">Back to Contents</a>

### *StreamMap* <a name="ex8_streammap"/>


In [11]:
s = (interleaveS (fmap Left s2) (fmap Right s1))

assertion_sm17 = and [eqSM ((sm ||| sm') >>> sm'') ((sm >>> sm'') ||| (sm' >>> sm'')) s
                      | sm <- sms, sm' <- sms, sm'' <- sms]

main :: IO ()
main = do
    putStrLn "Assertions:"
    print assertion_sm17

main


Assertions:
False

<a href="#contents">Back to Contents</a>

## Exercise 9 <a name="ex9"/>

Given the following definition, which adds string-valued exceptions to an arrow:

`newtype Except α β γ = E (α β (Either String γ))`

Define the following instance:

`instance ArrowChoice α ⇒ Arrow (Except α)`


In [12]:
newtype Except a b c = E {runE :: a b (Either String c)}

instance ArrowChoice a => Arrow (Except a) where
    pure f      = E (pure (\x -> (Right (f x))))
    
--    E f >>> E g = E (f >>> (right g) >>> (pure (\p -> case p of
--                                                        Left  s -> Left s
--                                                        Right x -> x)))
--    E f >>> E g = E (f >>> (right g) >>> (right (pure untag)))
    E f >>> E g = E (f >>> right (g >>> pure untag))
      where untag (Right y) = y



--    first f     = E (first (runE f) >>> (pure (\(p, y) -> case p of
--                                                            Left s  -> Left s
--                                                            Right x -> Right (x, y))))
    first (E f) = E (first f >>> pure distr >>> left (pure fst))


<a href="#contents">Back to Contents</a>

## Exercise 10 <a name="ex10"/>

Prove the functor axiom for first in the arrow defined in the previous exercise. This requires the distributivity axiom.

```
distr :: (Either α β, γ) → Either (α, γ) (β, γ)  
distr (Left  a, c) = Left  (a, c)
distr (Right b, c) = Right (b, c)
```

**distribution:** &nbsp; `first (left f ) ⋙ pure distr = pure distr ⋙ left (first f )`

**To Prove:** &nbsp; `first (f ⋙ g) = first f ⋙ first g`

`first (E f ⋙ E g)                                                       =` &nbsp; &nbsp; {definition of (⋙), for *Exception*}

`first (E (f ⋙ right (g ⋙ pure untag)))                                 =` &nbsp; &nbsp; {definition of *first*, for *Exception*}

`E (first (f ⋙ right (g ⋙ pure untag)) ⋙ pure distr ⋙ left (pure fst)) =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

` =` &nbsp; &nbsp; {}

`E (first f ⋙ pure distr ⋙ left (pure fst) ⋙
  right ((first g ⋙ pure distr ⋙ left (pure fst)) ⋙ pure untag))       =` &nbsp; &nbsp; {definition of (⋙), for *Exception*}

`E (first f ⋙ pure distr ⋙ left (pure fst)) ⋙
  E (first g ⋙ pure distr ⋙ left (pure fst))                            =` &nbsp; &nbsp; {definition of *first*, for *Exception*}

`first (E f) ⋙ first (E g) =`
&#9633;


<a href="#contents">Back to Contents</a>

## Unanswered Questions: <a name="questions"/>

1. Can pattern matching be used, in lambda definitions, or must you use a named function, if you want to pattern match?
1. How does Ross get away with the *untag* and *mirror* functions, without requiring both of the type parameters of the *Either* to be the same?


<a href="#contents">Back to Contents</a>